In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

from scipy.misc import imread, imsave
import cPickle as pickle
import yaml
import collections
from tqdm import tqdm
import time
from scipy.ndimage.interpolation import zoom

# Loading and interpreting results

In [2]:
base = '/media/michael/Engage/data/audio/bird_audio_detection_challenge/'

if 0:

    import h5py
    h5path = base + '/results/master/predictions/golden_model_second_3_2.h5'
    f = h5py.File(h5path)
    
    res = np.array(f['results'])
    fname_to_res = {fname.split('.')[0]:r for fname, r in zip(fnames, res)}
    
    names = [fname.split('.')[0] for fname in fnames]
    orig_fnames = set('_'.join(name.split('_')[:-1]) for name in names)
else:
    import pandas as pd
    res = pd.read_csv(base + 'results//as_provided_by_author//prediction_first.csv')

In [3]:
fname_to_res = {}
for row in res.iterrows():
    fname_to_res[row[1]['fname']] = row[1]['Unnamed: 1']
    
orig_fnames = set('_'.join(name.split('_')[:-2]) for name in fname_to_res)

In [20]:
aggragated_preds = {}

# we're going to chunk up each golden audio file into 10second bits 
overlap_time = 0
chunklength = 1

for orig_fname in orig_fnames:
    
    this_res = np.zeros(60.0 / chunklength)
    for start_idx, start_time in enumerate(np.arange(0, 60, chunklength)):
        
        name = orig_fname + '_%05.2f' % start_time 
        name = name.replace('.', '_')
        
        # note - assuming one second gaps
        this_res[start_idx] = fname_to_res[name]
        
    aggragated_preds[orig_fname] = np.zeros_like(this_res)

    aggragated_preds[orig_fname] = this_res

/home/michael/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


# Saving results in our format

In [21]:
# load gt
gts = {}
gt_dir = '/media/michael/Engage/data/audio/alison_data/golden_set/extracted/annotations/'
for fname in aggragated_preds:
    gts[fname] = pickle.load(open(gt_dir + fname + '.pkl'))[0]['biotic']

In [22]:
savedir = '/media/michael/Engage/data/audio/alison_data/golden_set/predictions/warblr_challenge_baseline/biotic/per_file_predictions/'

for fname in aggragated_preds.keys():
    pred = aggragated_preds[fname]
    resized_pred = zoom(pred.astype(float), 1319.0 / pred.shape[0], order=1)
    resized_pred = np.vstack((1.0 - resized_pred, resized_pred)).T
    resized_gt = zoom(gts[fname].astype(float), 1319.0 / gts[fname].shape[0], order=1)

    with open(savedir + fname + '.pkl', 'w') as f:
        pickle.dump((resized_gt, resized_pred), f, -1)